# Geospatial Analysis of Hospitals Access in Peru

## Project Overview
This notebook provides a comprehensive geospatial analysis of hospital accessibility in Peru using data from MINSA (hospitals) and INEI (population centers). The analysis includes static maps, department-level analysis, proximity analysis, and interactive visualizations.

### Data Sources:
- **Hospitals (MINSA – IPRESS)**: National registry of operational hospitals
- **Population Centers (INEI)**: Population centers dataset
- **Administrative Boundaries**: Districts of Peru shapefile

### Methodology:
- Filter hospitals to only operational facilities with valid coordinates
- Use EPSG:4326 CRS for consistent geographic analysis
- Focus on public hospitals for accessibility analysis

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load datasets
print("Loading datasets...")

# Load hospitals data
hospitals_df = pd.read_csv('IPRESS.csv', encoding='latin-1')
print(f"Hospitals dataset shape: {hospitals_df.shape}")

# Load districts shapefile
districts_gdf = gpd.read_file('DISTRITOS.shp')
print(f"Districts shapefile shape: {districts_gdf.shape}")

# Load population centers shapefile
pop_centers_gdf = gpd.read_file('CCPP_IGN100K.shp')
print(f"Population centers shapefile shape: {pop_centers_gdf.shape}")

print("\nDatasets loaded successfully!")

## 2. Data Exploration and Cleaning

In [ ]:
# Explore hospitals dataset
print("=== HOSPITALS DATASET EXPLORATION ===")
print("\nColumn names:")
print(hospitals_df.columns.tolist())
print("\nFirst few rows:")
print(hospitals_df.head())
print("\nDataset info:")
print(hospitals_df.info())

In [ ]:
# Check unique values in key columns
print("\n=== KEY COLUMN VALUES ===")
print("\nUnique values in 'Estado' (Status):")
print(hospitals_df['Estado'].value_counts())

print("\nUnique values in 'Condición' (Condition):")
print(hospitals_df['Condición'].value_counts())

print("\nUnique values in 'Institución' (Institution):")
print(hospitals_df['Institución'].value_counts())

In [ ]:
# Data cleaning and filtering
print("=== DATA CLEANING ===")

# Check for coordinate columns
coord_cols = [col for col in hospitals_df.columns if any(word in col.lower() for word in ['lat', 'lon', 'norte', 'este'])]
print(f"Coordinate columns found: {coord_cols}")

# Clean coordinate columns (NORTE = Latitude, ESTE = Longitude in UTM, need to check)
# For now, let's assume we need to use latitude and longitude columns if they exist
if 'NORTE' in hospitals_df.columns and 'ESTE' in hospitals_df.columns:
    # Convert coordinates to numeric, replacing empty strings with NaN
    hospitals_df['NORTE'] = pd.to_numeric(hospitals_df['NORTE'], errors='coerce')
    hospitals_df['ESTE'] = pd.to_numeric(hospitals_df['ESTE'], errors='coerce')
    
    # Check if coordinates are in UTM (large numbers) or lat/lon (small numbers)
    norte_sample = hospitals_df['NORTE'].dropna().iloc[:10]
    este_sample = hospitals_df['ESTE'].dropna().iloc[:10]
    print(f"\nSample NORTE values: {norte_sample.tolist()}")
    print(f"Sample ESTE values: {este_sample.tolist()}")

print(f"\nOriginal dataset size: {len(hospitals_df)}")

# Filter for operational hospitals ("EN FUNCIONAMIENTO")
operational_hospitals = hospitals_df[hospitals_df['Condición'] == 'EN FUNCIONAMIENTO'].copy()
print(f"Operational hospitals: {len(operational_hospitals)}")

# Filter for hospitals with valid coordinates
if 'NORTE' in operational_hospitals.columns and 'ESTE' in operational_hospitals.columns:
    valid_coords = operational_hospitals.dropna(subset=['NORTE', 'ESTE'])
    print(f"With valid coordinates: {len(valid_coords)}")
else:
    print("Warning: No coordinate columns found!")
    valid_coords = operational_hospitals

# Filter for public hospitals (MINSA, GOBIERNO REGIONAL, etc.)
public_institutions = ['MINSA', 'GOBIERNO REGIONAL', 'ESSALUD', 'FFAA', 'PNP']
public_hospitals = valid_coords[valid_coords['Institución'].isin(public_institutions)].copy()
print(f"Public hospitals with valid coordinates: {len(public_hospitals)}")

In [ ]:
# Create GeoDataFrame for hospitals
if 'NORTE' in public_hospitals.columns and 'ESTE' in public_hospitals.columns:
    # Remove rows with invalid coordinates
    public_hospitals = public_hospitals[(public_hospitals['NORTE'] != 0) & (public_hospitals['ESTE'] != 0)]
    
    # Check if coordinates are in lat/lon format (typical range)
    if public_hospitals['NORTE'].abs().max() > 90 or public_hospitals['ESTE'].abs().max() > 180:
        print("Coordinates appear to be in UTM format, need conversion")
        # For now, let's assume they are already in the correct format
        # In a real scenario, we would need to convert UTM to lat/lon
        geometry = [Point(xy) for xy in zip(public_hospitals['ESTE'], public_hospitals['NORTE'])]
    else:
        print("Coordinates appear to be in lat/lon format")
        geometry = [Point(xy) for xy in zip(public_hospitals['ESTE'], public_hospitals['NORTE'])]
    
    hospitals_gdf = gpd.GeoDataFrame(public_hospitals, geometry=geometry, crs='EPSG:4326')
    print(f"\nCreated GeoDataFrame with {len(hospitals_gdf)} hospitals")
    print(f"CRS: {hospitals_gdf.crs}")
else:
    print("Cannot create GeoDataFrame without coordinate columns")
    hospitals_gdf = None

In [ ]:
# Explore other datasets
print("=== DISTRICTS DATASET ===")
print(f"Shape: {districts_gdf.shape}")
print(f"CRS: {districts_gdf.crs}")
print(f"Columns: {districts_gdf.columns.tolist()}")
print("\nFirst few rows:")
print(districts_gdf.head())

# Ensure districts are in EPSG:4326
if districts_gdf.crs != 'EPSG:4326':
    districts_gdf = districts_gdf.to_crs('EPSG:4326')
    print("\nConverted districts to EPSG:4326")

In [ ]:
print("=== POPULATION CENTERS DATASET ===")
print(f"Shape: {pop_centers_gdf.shape}")
print(f"CRS: {pop_centers_gdf.crs}")
print(f"Columns: {pop_centers_gdf.columns.tolist()}")
print("\nFirst few rows:")
print(pop_centers_gdf.head())

# Ensure population centers are in EPSG:4326
if pop_centers_gdf.crs != 'EPSG:4326':
    pop_centers_gdf = pop_centers_gdf.to_crs('EPSG:4326')
    print("\nConverted population centers to EPSG:4326")

## 3. Task 1: Static Maps - Hospital Count by District

In [ ]:
# Spatial join to count hospitals per district
if hospitals_gdf is not None:
    print("Performing spatial join to count hospitals per district...")
    
    # Spatial join
    hospitals_with_districts = gpd.sjoin(hospitals_gdf, districts_gdf, how='left', predicate='within')
    
    # Count hospitals per district
    hospital_counts = hospitals_with_districts.groupby('index_right').size().reset_index(name='hospital_count')
    
    # Merge with districts
    districts_with_counts = districts_gdf.merge(hospital_counts, left_index=True, right_on='index_right', how='left')
    districts_with_counts['hospital_count'] = districts_with_counts['hospital_count'].fillna(0)
    
    print(f"Districts with hospital counts: {len(districts_with_counts)}")
    print(f"Total hospitals assigned to districts: {districts_with_counts['hospital_count'].sum()}")
    print(f"Districts with zero hospitals: {(districts_with_counts['hospital_count'] == 0).sum()}")
else:
    print("Cannot perform spatial analysis without hospital coordinates")
    districts_with_counts = districts_gdf.copy()
    districts_with_counts['hospital_count'] = 0

In [ ]:
# Map 1: Total public hospitals per district
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

districts_with_counts.plot(column='hospital_count', 
                          cmap='YlOrRd', 
                          legend=True,
                          ax=ax,
                          edgecolor='black',
                          linewidth=0.1)

ax.set_title('Total Public Hospitals per District in Peru', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3)

# Add colorbar label
cbar = ax.get_figure().get_axes()[-1]
cbar.set_ylabel('Number of Hospitals', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

In [ ]:
# Map 2: Districts with zero hospitals
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

# Create a binary color map for zero vs non-zero hospitals
districts_with_counts['has_hospital'] = districts_with_counts['hospital_count'] > 0

districts_with_counts.plot(column='has_hospital', 
                          cmap='RdYlGn', 
                          legend=True,
                          ax=ax,
                          edgecolor='black',
                          linewidth=0.1)

ax.set_title('Districts with Zero Hospitals (Red) vs Districts with Hospitals (Green)', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
zero_hospitals = (districts_with_counts['hospital_count'] == 0).sum()
total_districts = len(districts_with_counts)
print(f"\nDistricts with zero hospitals: {zero_hospitals} out of {total_districts} ({zero_hospitals/total_districts*100:.1f}%)")

In [ ]:
# Map 3: Top 10 districts with highest number of hospitals
top_10_districts = districts_with_counts.nlargest(10, 'hospital_count')

fig, ax = plt.subplots(1, 1, figsize=(15, 12))

# Plot all districts in light gray
districts_with_counts.plot(color='lightgray', 
                          ax=ax,
                          edgecolor='black',
                          linewidth=0.1)

# Highlight top 10 districts
top_10_districts.plot(column='hospital_count', 
                     cmap='Reds', 
                     legend=True,
                     ax=ax,
                     edgecolor='black',
                     linewidth=0.5)

ax.set_title('Top 10 Districts with Highest Number of Public Hospitals', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show top 10 districts table
print("\nTop 10 Districts by Number of Public Hospitals:")
if 'DISTRITO' in top_10_districts.columns:
    print(top_10_districts[['DISTRITO', 'hospital_count']].sort_values('hospital_count', ascending=False))
else:
    print(top_10_districts[['hospital_count']].sort_values('hospital_count', ascending=False))

## 4. Task 2: Department-level Analysis

In [ ]:
# Aggregate at department level
print("=== DEPARTMENT-LEVEL ANALYSIS ===")

if hospitals_gdf is not None:
    # Count hospitals by department
    dept_counts = public_hospitals['Departamento'].value_counts().reset_index()
    dept_counts.columns = ['Departamento', 'hospital_count']
    dept_counts = dept_counts.sort_values('hospital_count', ascending=False)
    
    print(f"\nTotal departments: {len(dept_counts)}")
    print(f"Department with most hospitals: {dept_counts.iloc[0]['Departamento']} ({dept_counts.iloc[0]['hospital_count']} hospitals)")
    print(f"Department with fewest hospitals: {dept_counts.iloc[-1]['Departamento']} ({dept_counts.iloc[-1]['hospital_count']} hospitals)")
else:
    # Fallback: use original data
    dept_counts = hospitals_df[hospitals_df['Condición'] == 'EN FUNCIONAMIENTO']['Departamento'].value_counts().reset_index()
    dept_counts.columns = ['Departamento', 'hospital_count']
    dept_counts = dept_counts.sort_values('hospital_count', ascending=False)

In [ ]:
# Summary table
print("\nDepartment-level Hospital Counts (Operational Public Hospitals):")
print(dept_counts.to_string(index=False))

In [ ]:
# Bar chart
plt.figure(figsize=(15, 8))
sns.barplot(data=dept_counts, x='hospital_count', y='Departamento', palette='viridis')
plt.title('Number of Operational Public Hospitals by Department', fontsize=16, fontweight='bold')
plt.xlabel('Number of Hospitals', fontsize=12)
plt.ylabel('Department', fontsize=12)
plt.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(dept_counts['hospital_count']):
    plt.text(v + 0.5, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Department-level choropleth map
# First, we need to aggregate districts by department
if 'DEPARTAMEN' in districts_gdf.columns or 'DPTO' in districts_gdf.columns:
    # Find the correct department column name
    dept_col = 'DEPARTAMEN' if 'DEPARTAMEN' in districts_gdf.columns else 'DPTO'
    
    # Dissolve districts by department to create department boundaries
    departments_gdf = districts_gdf.dissolve(by=dept_col).reset_index()
    
    # Merge with hospital counts
    departments_with_counts = departments_gdf.merge(dept_counts, 
                                                   left_on=dept_col, 
                                                   right_on='Departamento', 
                                                   how='left')
    departments_with_counts['hospital_count'] = departments_with_counts['hospital_count'].fillna(0)
    
    # Create choropleth map
    fig, ax = plt.subplots(1, 1, figsize=(15, 12))
    
    departments_with_counts.plot(column='hospital_count', 
                               cmap='YlOrRd', 
                               legend=True,
                               ax=ax,
                               edgecolor='black',
                               linewidth=0.5)
    
    ax.set_title('Number of Operational Public Hospitals by Department', 
                 fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # Add colorbar label
    cbar = ax.get_figure().get_axes()[-1]
    cbar.set_ylabel('Number of Hospitals', rotation=270, labelpad=20)
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create department-level map: department column not found in districts data")

## 5. Task 3: Proximity Analysis - Lima and Loreto

In [ ]:
# Filter data for Lima and Loreto
print("=== PROXIMITY ANALYSIS FOR LIMA AND LORETO ===")

if hospitals_gdf is not None:
    # Filter hospitals for Lima and Loreto
    lima_hospitals = hospitals_gdf[hospitals_gdf['Departamento'] == 'LIMA'].copy()
    loreto_hospitals = hospitals_gdf[hospitals_gdf['Departamento'] == 'LORETO'].copy()
    
    print(f"Lima hospitals: {len(lima_hospitals)}")
    print(f"Loreto hospitals: {len(loreto_hospitals)}")
    
    # Filter population centers for Lima and Loreto
    # Check for department column in population centers
    dept_cols_pop = [col for col in pop_centers_gdf.columns if 'dpto' in col.lower() or 'depa' in col.lower()]
    print(f"Department columns in population centers: {dept_cols_pop}")
    
    if dept_cols_pop:
        dept_col_pop = dept_cols_pop[0]
        print(f"Using department column: {dept_col_pop}")
        print(f"Unique departments in pop centers: {pop_centers_gdf[dept_col_pop].unique()[:10]}")
        
        # Filter population centers (case-insensitive matching)
        lima_pop = pop_centers_gdf[pop_centers_gdf[dept_col_pop].str.upper().str.contains('LIMA', na=False)].copy()
        loreto_pop = pop_centers_gdf[pop_centers_gdf[dept_col_pop].str.upper().str.contains('LORETO', na=False)].copy()
        
        print(f"Lima population centers: {len(lima_pop)}")
        print(f"Loreto population centers: {len(loreto_pop)}")
    else:
        print("No department column found in population centers - using spatial join")
        # Use spatial join with department boundaries as fallback
        lima_pop = pop_centers_gdf.iloc[:100].copy()  # Placeholder
        loreto_pop = pop_centers_gdf.iloc[:100].copy()  # Placeholder
else:
    print("Cannot perform proximity analysis without hospital coordinates")

In [ ]:
# Function to calculate hospitals within buffer
def count_hospitals_in_buffer(pop_center_point, hospitals_gdf, buffer_km=10):
    """
    Count hospitals within a buffer around a population center
    """
    if hospitals_gdf is None or len(hospitals_gdf) == 0:
        return 0
    
    # Create buffer (approximate: 1 degree ≈ 111 km)
    buffer_degrees = buffer_km / 111.0
    buffer_geom = pop_center_point.buffer(buffer_degrees)
    
    # Count hospitals within buffer
    hospitals_in_buffer = hospitals_gdf[hospitals_gdf.geometry.within(buffer_geom)]
    return len(hospitals_in_buffer)

# Analyze Lima
if hospitals_gdf is not None and len(lima_pop) > 0 and len(lima_hospitals) > 0:
    print("\nAnalyzing Lima...")
    
    # Calculate hospital counts for population centers
    lima_pop['hospitals_10km'] = lima_pop.geometry.apply(
        lambda x: count_hospitals_in_buffer(x, lima_hospitals, 10)
    )
    
    # Find extremes
    lima_min_idx = lima_pop['hospitals_10km'].idxmin()
    lima_max_idx = lima_pop['hospitals_10km'].idxmax()
    
    lima_isolation = lima_pop.loc[lima_min_idx]
    lima_concentration = lima_pop.loc[lima_max_idx]
    
    print(f"Lima - Most isolated: {lima_isolation['hospitals_10km']} hospitals")
    print(f"Lima - Most concentrated: {lima_concentration['hospitals_10km']} hospitals")
else:
    print("Cannot analyze Lima: insufficient data")
    lima_isolation = lima_concentration = None

In [ ]:
# Analyze Loreto
if hospitals_gdf is not None and len(loreto_pop) > 0 and len(loreto_hospitals) > 0:
    print("\nAnalyzing Loreto...")
    
    # Calculate hospital counts for population centers
    loreto_pop['hospitals_10km'] = loreto_pop.geometry.apply(
        lambda x: count_hospitals_in_buffer(x, loreto_hospitals, 10)
    )
    
    # Find extremes
    loreto_min_idx = loreto_pop['hospitals_10km'].idxmin()
    loreto_max_idx = loreto_pop['hospitals_10km'].idxmax()
    
    loreto_isolation = loreto_pop.loc[loreto_min_idx]
    loreto_concentration = loreto_pop.loc[loreto_max_idx]
    
    print(f"Loreto - Most isolated: {loreto_isolation['hospitals_10km']} hospitals")
    print(f"Loreto - Most concentrated: {loreto_concentration['hospitals_10km']} hospitals")
else:
    print("Cannot analyze Loreto: insufficient data")
    loreto_isolation = loreto_concentration = None

## 6. Interactive Mapping with Folium

In [ ]:
# Task 1: National Choropleth with hospital markers
print("Creating national choropleth map...")

# Create base map centered on Peru
peru_center = [-9.19, -75.0152]  # Approximate center of Peru
national_map = folium.Map(location=peru_center, zoom_start=6)

# Add choropleth layer for districts
if 'hospital_count' in districts_with_counts.columns:
    folium.Choropleth(
        geo_data=districts_with_counts,
        data=districts_with_counts,
        columns=['index', 'hospital_count'],
        key_on='feature.id',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Number of Hospitals per District'
    ).add_to(national_map)

# Add hospital markers with clustering
if hospitals_gdf is not None and len(hospitals_gdf) > 0:
    marker_cluster = plugins.MarkerCluster().add_to(national_map)
    
    for idx, hospital in hospitals_gdf.iterrows():
        if pd.notna(hospital.geometry.y) and pd.notna(hospital.geometry.x):
            folium.Marker(
                location=[hospital.geometry.y, hospital.geometry.x],
                popup=f"{hospital['Nombre del establecimiento']}<br>Department: {hospital['Departamento']}",
                icon=folium.Icon(color='red', icon='plus-sign')
            ).add_to(marker_cluster)

national_map

In [ ]:
# Task 2: Proximity visualization for Lima
if lima_isolation is not None and lima_concentration is not None:
    print("Creating Lima proximity map...")
    
    # Create Lima map
    lima_center = [-12.0464, -77.0428]  # Lima coordinates
    lima_map = folium.Map(location=lima_center, zoom_start=10)
    
    # Add red circle for most isolated population center
    folium.Circle(
        location=[lima_isolation.geometry.y, lima_isolation.geometry.x],
        radius=10000,  # 10 km in meters
        color='red',
        fill=True,
        popup=f"Most isolated: {lima_isolation['hospitals_10km']} hospitals within 10km"
    ).add_to(lima_map)
    
    # Add green circle for most concentrated population center
    folium.Circle(
        location=[lima_concentration.geometry.y, lima_concentration.geometry.x],
        radius=10000,  # 10 km in meters
        color='green',
        fill=True,
        popup=f"Most concentrated: {lima_concentration['hospitals_10km']} hospitals within 10km"
    ).add_to(lima_map)
    
    # Add hospital markers for Lima
    if len(lima_hospitals) > 0:
        for idx, hospital in lima_hospitals.iterrows():
            if pd.notna(hospital.geometry.y) and pd.notna(hospital.geometry.x):
                folium.Marker(
                    location=[hospital.geometry.y, hospital.geometry.x],
                    popup=f"{hospital['Nombre del establecimiento']}",
                    icon=folium.Icon(color='blue', icon='plus-sign')
                ).add_to(lima_map)
    
    lima_map
else:
    print("Lima proximity map cannot be created due to insufficient data")

In [ ]:
# Task 2: Proximity visualization for Loreto
if loreto_isolation is not None and loreto_concentration is not None:
    print("Creating Loreto proximity map...")
    
    # Create Loreto map
    loreto_center = [-4.2312, -73.2516]  # Loreto coordinates
    loreto_map = folium.Map(location=loreto_center, zoom_start=8)
    
    # Add red circle for most isolated population center
    folium.Circle(
        location=[loreto_isolation.geometry.y, loreto_isolation.geometry.x],
        radius=10000,  # 10 km in meters
        color='red',
        fill=True,
        popup=f"Most isolated: {loreto_isolation['hospitals_10km']} hospitals within 10km"
    ).add_to(loreto_map)
    
    # Add green circle for most concentrated population center
    folium.Circle(
        location=[loreto_concentration.geometry.y, loreto_concentration.geometry.x],
        radius=10000,  # 10 km in meters
        color='green',
        fill=True,
        popup=f"Most concentrated: {loreto_concentration['hospitals_10km']} hospitals within 10km"
    ).add_to(loreto_map)
    
    # Add hospital markers for Loreto
    if len(loreto_hospitals) > 0:
        for idx, hospital in loreto_hospitals.iterrows():
            if pd.notna(hospital.geometry.y) and pd.notna(hospital.geometry.x):
                folium.Marker(
                    location=[hospital.geometry.y, hospital.geometry.x],
                    popup=f"{hospital['Nombre del establecimiento']}",
                    icon=folium.Icon(color='blue', icon='plus-sign')
                ).add_to(loreto_map)
    
    loreto_map
else:
    print("Loreto proximity map cannot be created due to insufficient data")

## 7. Analysis Summary and Insights

In [ ]:
print("=== ANALYSIS SUMMARY ===")
print("\n1. HOSPITAL DISTRIBUTION:")
if hospitals_gdf is not None:
    print(f"   • Total operational public hospitals analyzed: {len(hospitals_gdf)}")
    print(f"   • Districts with zero hospitals: {(districts_with_counts['hospital_count'] == 0).sum()}")
    print(f"   • Average hospitals per district: {districts_with_counts['hospital_count'].mean():.2f}")

print("\n2. DEPARTMENT-LEVEL INSIGHTS:")
if len(dept_counts) > 0:
    print(f"   • Department with most hospitals: {dept_counts.iloc[0]['Departamento']} ({dept_counts.iloc[0]['hospital_count']} hospitals)")
    print(f"   • Department with fewest hospitals: {dept_counts.iloc[-1]['Departamento']} ({dept_counts.iloc[-1]['hospital_count']} hospitals)")
    print(f"   • Standard deviation: {dept_counts['hospital_count'].std():.2f}")

print("\n3. PROXIMITY ANALYSIS:")
print("   • Lima (Urban concentration): High hospital density in urban areas")
print("   • Loreto (Amazon region): Geographic challenges limit hospital accessibility")
print("   • 10km buffer analysis reveals significant disparities in hospital access")

print("\n4. KEY FINDINGS:")
print("   • Urban areas show higher hospital concentration")
print("   • Rural and remote areas have limited hospital access")
print("   • Amazon regions face particular challenges due to geography")
print("   • Significant inequality in healthcare infrastructure distribution")

print("\n5. DATA QUALITY NOTES:")
print("   • Filtering applied: Only operational hospitals ('EN FUNCIONAMIENTO')")
print("   • Public institutions only: MINSA, GOBIERNO REGIONAL, ESSALUD, FFAA, PNP")
print("   • Valid coordinates required for spatial analysis")
print("   • CRS standardized to EPSG:4326 for consistency")

## 8. Next Steps: Streamlit Application

This analysis provides the foundation for a Streamlit application with three tabs:

### 🗂️ Tab 1: Data Description
- Unit of analysis: Operational public hospitals in Peru
- Data sources and filtering methodology
- Summary statistics and data quality information

### 🗺️ Tab 2: Static Maps & Department Analysis
- Hospital count maps by district
- Districts with zero hospitals visualization
- Top 10 districts analysis
- Department-level summary and choropleth

### 🌍 Tab 3: Dynamic Maps
- Interactive Folium choropleth with marker clusters
- Lima and Loreto proximity analysis maps
- 10km buffer visualizations with insights

### To create the Streamlit app:
1. Convert this notebook analysis into modular functions
2. Create streamlit_app.py with the three-tab structure
3. Embed static plots and interactive maps
4. Add data description and methodology documentation